# Big Data y Machine Learning (UBA) 2025
## Clase 2 - Parte III - Visualización de NaNs
En esta tutorial, simularemos datos con algunas variables muy parecidas a la Encuesta Permanente de Hogares (EPH). También, simularemos valores faltantes, como contabilizarlos y visualizarlos para una limpieza posterior de la base de datos. Este paso es muy util en el momento de **procesamiento y preparación** de la muestra analítica antes de correr cualquier modelo predictivo o método no supervisado.
Para dicha visualización de NaN usaremos el modulo de `seaborn`, también muy utilizada en procesamiento y visualización de datos en Python. Para más información ver [seaborn](https://seaborn.pydata.org/)

In [ ]:
# Primero, installamos el paquete
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Paso 1: Simulamos los datos

In [ ]:
# Fijamos la semilla para obtener resultados reproducibles
np.random.seed(123)

In [ ]:
# Creamos un arreglo de números enteros del 1 al 1000 para identificar a 1000 personas
personas = np.arange(1, 1000)

In [ ]:
# Creamos un arreglo de años desde 2005 hasta 2025
anios = np.arange(2005, 2025)

In [ ]:
# Creamos un DataFrame con todas las combinaciones posibles de personas y años
panel = pd.DataFrame([(persona, anio) for persona in personas for anio in anios], 
                     columns=['id_persona', 'anio'])

In [ ]:
# Generamos edades aleatorias entre 18 y 64 años para cada observación
panel['edad'] = np.random.randint(18, 65, size=len(panel))

In [ ]:
# Generamos alturas aleatorias con una distribución normal, media de 170cm y desviación estándar de 10cm
panel['altura'] = np.round(np.random.normal(170, 10, size=len(panel)), 1)

In [ ]:
# Creamos una variable dummy para secundaria completa (0 o 1), con probabilidad del 70% para el valor 1
panel['secundaria_completa'] = np.random.choice([0, 1], size=len(panel), p=[0.3, 0.7])

In [ ]:
# Generamos salarios aleatorios siguiendo una distribución normal con media de 60,000 y desviación estándar de 15,000
panel['salario'] = np.round(np.random.normal(60000, 15000, size=len(panel)), 2)

In [ ]:
panel.shape

In [ ]:
panel.info(verbose=True)

#### Simulamos los valores faltantes (NaN)

In [ ]:
# Introducimos valores faltantes (NaN) en cada columna, según los porcentajes especificados
for col, prop in {'edad': 0.10, 'altura': 0.23, 'secundaria_completa': 0.05, 'salario': 0.30}.items():
    # Calculamos la cantidad de valores faltantes a introducir en la columna actual
    n_missing = int(np.floor(prop * len(panel)))
    # Seleccionamos índices aleatorios donde insertaremos los valores faltantes
    missing_indices = np.random.choice(panel.index, size=n_missing, replace=False)
    # Asignamos valores faltantes (NaN) a los índices seleccionados en la columna actual
    panel.loc[missing_indices, col] = np.nan

#### Base de datos "raw"

Ahora, recuerden lo que vimos en tutoriales para inspeccionar la base de datos

In [ ]:
# Visualizamos los datos
panel.sample(10)

In [ ]:
panel.info(verbose = True) #con esto, ya podemos ver la canrisas de valores no nulos por variable

Veamos la estadistica descriptiva de la base de datos raw

In [ ]:
panel.describe().round(2)

### Paso 2: Contabilizamos y visualizamos los NaN

In [ ]:
# Calcular el porcentaje de datos no nulos por año
percentage_data = panel.groupby('anio').apply(lambda x: x.notnull().mean() * 100)

In [ ]:
# Eliminar las columnas innecesarias antes de graficar
percentage_data = percentage_data.drop(columns=['anio'])  # Eliminar 'anio' del DataFrame calculado

In [ ]:
# Crear el heatmap sin etiquetas en las celdas
plt.figure(figsize=(12, 8))
sns.heatmap(percentage_data.T, annot=False, cmap="coolwarm_r",cbar_kws={'label': 'Porcentaje de datos no-nulos'})
plt.title("Figura 1. Porcentaje de información disponible por año")
plt.xlabel("Año")
plt.xticks(rotation=45, ha='right')
plt.ylabel("Variables")
plt.yticks(rotation=0)
plt.show()